# C++学习笔记

本文主要介绍C++中跟类有关的知识点。

## 第十章 对象和类

这一章主要是简单的介绍了C++中类的相关概念，主要的知识点有以下几点：

1. 在类中定义的方法都是默认内联的，这些方法也可以显式地内联定义。同样也可以不采用内联定义。如果使用内联定义一定要在头文件中，否则会引发错误。
2. 使用`const`关键字修饰的对象调用的方法也必须使用`const`关键字修饰，使用`const`关键字修饰的方法不能修改类的任何内容。
3. 在类的内部使用`this`关键字指代这个类对象的指针，`this`指针指向类对象本身，如果要返回类对象而非指针，那么应该使用`*`运算符`*this`。
4. 类中往往需要使用一些常量来规定例如数组的长度或者栈的大小，这种情况下可以使用作用域内的枚举或者静态`const`变量来实现。
5. C++11中可以使用`{}`来实例化对象，并不一定要显式调用构造方法。

### 内联的类方法

在C++中如果一个函数或方法声明为内联的，那么这个方法就需要在每个要使用它的程序文件中都定义一次，这种特性就决定了内联方法或者函数最好在头文件中被定义，因为每当头文件被`#include`的时候就会执行一次内联函数或方法的定义。

C++中在类中声明的方法默认就是内联的：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person(){
        this->name = "unknown";
        this->age = 0;
        this->gen = male;
    }
    void show() const{
        cout << "The person " << this->name
             << "'s age is " << this->age
             << ", his/her gender is " << this->gen << endl;
    }
};
```
以上代码包含了`Person`类的构造方法和一个普通的成员方法，这两个方法都是以默认形式实现的，这说明这两个方法都是内联方法。这也就是为什么这个类能够在`person.h`这样的头文件中被定义而不用担心方法重复定义的问题，因为内联函数或方法本身就是要在每个使用的程序文件中都定义一次。另外，出了这种默认的写法，内敛方法也可以显示地去定义：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person();
    void show() const;
};

inline Person::Person(){//inline关键字需要显式给出
    this->name = "unknown";
    this->age = 0;
    this->gen = male;
}

inline void Person::show() const{
    cout << "The person " << this->name
         << "'s age is " << this->age
         << ", his/her gender is " << this->gen << endl;
}
```

最后，类的成员方法还可以不以内联的方式来定义，这种定义方式就不能在头文件中了，因为这将会引发方法重复定义的错误：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person();
    void show() const;
};

//person.cpp
void Person::show(){
    cout << "The person " << this->name
         << "'s age is " << this->age
         << ", his/her gender is " << this->gen << endl;
}
Person::Person(){
    this->name = "unknown";
    this->age = 0;
    this->gen = male;
}
```

### `const`关键字修饰的实例

使用`const`关键字修饰一个实例意味着这个实例是不可变的，这个不可变的实例调用的方法也应该是`const`关键字修饰的，否则就会印发错误：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person(){
        this->name = "unknown";
        this->age = 0;
        this->gen = male;
    }
    void show() const{
        cout << "The person " << this->name
             << "'s age is " << this->age
             << ", his/her gender is " << this->gen << endl;
    }
};

//main.cpp
#include "Person.h"
using namespace std;
const Person p = Person();
//如果show()方法没有使用const关键字修饰，那么就会引发错误
p.show();
```

### 类中的常量

通常情况下，在类中如果需要声明一个常量，那么通常会想到的做法是声明一个不可变的成员变量：`cosnt int size = 15;`，但是这种做法往往是行不通的，因为这种类型的普通成员变量都是在类已经实例化之后赋值，如果要使用这种常量作为类中数组或者栈的大小，那么编译器就会引发错误，因为数组或者栈的大小需要在类实例化之前就要确定，然而这种形式下类实例化之前变量还没有被赋值。解决这种问题的方案有两个：

1. 类作用域内部的枚举。
2. 静态`const`变量。

使用类作用域内部的枚举是声明一个匿名枚举：

```C++
//person.h
class person{
private:
    enum egg{five=12};
    double array[five];
}
```
另一种方式是使用静态`const`变量，这种静态变量的生存周期于类无关，它是和整个程序的生存周期相同的，因此在类实例化之前，相应的变量就已经赋值了。但是需要说明是这里的变量一定要使用`const`和`static`两个关键字修饰：

```C++
//person.h
class person{
private:
    static const int size = 15;
    double array[size];
}
```

### 对象的初始化

C++11中既可以使用传统的调用构造方法的形式来实例化对象，也可以使用C++中新定义的方法来实例化：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person(const std::string &name){
        this->name = name;
        this->age = 0;
        this->gen = male;
    }
    void show() const{
        cout << "The person " << this->name
             << "'s age is " << this->age
             << ", his/her gender is " << this->gen << endl;
    }
};

//main.cpp
const Person p = {"David"};
p.show();
```

这里虽然使用了`{}`的形式来初始化，但是在内部编译器还是根据传入的参数类型和个数（也就是特征标）来动态匹配调用相应的构造方法。

还有在命名空间中使用类和类的构造函数和析构函数相关知识比较简单，请看代码。

## 第十一章 使用类

本章介绍了类的一些基本使用，包括基本的运算符重载，基本的友元函数和类的类型强制转换，主要知识点由以下三个部分组成：

1. 运算符重载：运算符重载可以让程序员自定义运算符的行为，有三种形式对运算符进行重载：作为类方法的运算符重载，作为普通函数的运算符重载和作为友元函数的运算符重载。

2. 友元函数：友元函数为一个非成员函数提供了访问类私有成员的权限，通常友元函数是在类中声明一个以`friend`开头的函数原型，这个函数不是类的成员，它可以以隐式内联（也就是直接在类内部实现），显式内联（在类外部使用`inline`关键字实现）也可以使用普通的方式实现。需要说明的是：在实现时不需要加`friend`关键字。

3. C++中类的强制转换：可以基本数据类型转换为用户定义的类（需要有构造函数支持），这种转换默认是隐式的，为了防止犯错可以在相应构造函数前加上`explicit`关键字来强制定义显式类型转换。也可以将用户定义的类转换为基本数据类型，但是要定义并实现一个类方法：`operator typeName() const;`，同样为了防止犯错，可以使用`explicit`关键字来杜绝隐式类型转换。

### 运算符重载

运算符重载是C++中一个非常强大的地方，在C++中有三种方法重载运算符，它们分别是：

1. 作为类的成员方法重载运算符。

2. 作为普通函数重载运算符。

3. 作为友元函数重载运算符。

当运算符重载作为类的成员变量出现时：

```C++
//Date.h
namespace DATE{
    class Date{
    private:
        ...
    public:
        Date();
        ~Date();
        Date operator+(const Date &date);
        Date operator-(const Date &date);
    };
}
//Date.cpp
Date DATE::Date::operator-(const DATE::Date &date) {
    long interval = (this->million_seconds > date.million_seconds) ?
                        (this->million_seconds - date.million_seconds) : (date.million_seconds - this->million_seconds);
    Date res = Date(interval);
    return res;
}
Date DATE::Date::operator+(const DATE::Date &date) {
    Date res = Date(this->million_seconds + date.million_seconds);
    return res;
}
```

运算符重载也可以作为普通的函数出现，但是这种情况下类实例是没有权限访问类的私有变量的：

```C++
Date operator+(const Date &a, const Date &b){
    long interval = (a.million_seconds > b.million_seconds) ?
                        (a.million_seconds - b.million_seconds) : (b.million_seconds - a.million_seconds);
    Date res = Date(interval);
    return res;
}
Date operator-(const Date &a, const Date &b){
    Date res = Date(a.million_seconds + b.million_seconds);
    return res;
}
```
在使用这种形式重载运算符时需要指出的是：其参数列表中必须至少包含一个用户自定义的类型（即不能参数全是基本数据类型），否则无法通过编译。这种限制是为了防止用户通过重载运算符的形式改变基本数据类型的运算规则从而引起混乱。这一点在作为类成员变量的运算符重载身上并不存在，因为作为类成员的运算符重载的第一个操作数是自动传递的，本身就是用户自定义类型。

最后，运算符重载也可以作为友元函数出现。之所以要有这个设定是因为：作为普通函数出现的运算符重载可以任意定义操作数的位置但是类实例无法访问私有变量，作为类成员的运算符重载虽然类实例可以访问私有变量但是操作数的位置被限定死。友元函数的出现就解决了这两个难题：

```C++
//date.h
namespace DATE{
    class Date{
    private:
        ...
    public:
        Date();
        ~Date();
        friend std::ostream &operator<<(std::ostream &os, const Date &date);
    };
}
//date.cpp
std::ostream &operator<<(std::ostream &os, const Date &date) {
    if (date.format == format_24_hours) {
        os << std::setfill('0') << date.year << "-"
           << std::setw(2) << date.month << "-"
           << std::setw(2) << date.day << " "
           << std::setw(2) << date.hours << ":"
           << std::setw(2) << date.minutes << ":"
           << std::setw(2) << date.seconds;
    } else {
        os << std::setfill('0') << date.year << "-"
           << std::setw(2) << date.month << "-"
           << std::setw(2) << date.day << " "
           << std::setw(2) << ((date.hours > 12) ? (date.hours - 12) : date.hours) << ":"
           << std::setw(2) << date.minutes << ":"
           << std::setw(2) << date.seconds << ((date.hours >= 12) ? "pm" : "am");
    }
    return os;
}
```
上面代码就定义了一个友元函数实现的运算符重载，之所以`<<`运算符要通过友元函数来实现是因为：如果使用类成员方法来实现运算符重载，那么`Date`对象将处于第一操作数上，实际使用将会是`Date << cout`这与通常的表示不符。然而通过类成员函数实现的运算符重载不支持操作数位置的互换。使用普通函数实现运算符重载虽然可以更换操作数位置，但是无法访问`Date`私有变量。因此友元函数就成为了最佳选择。

最后要注意：定义运算符重载时一定要小心不要出现二义性。

### 友元函数

友元函数的知识点有以下几点：

1. 友元函数不是类的成员函数（也就是说无法通过类实例来调用它），但是必须要在类中声明，并且使用`friend`关键字。

2. 友元函数可以访问所在类的私有成员，同时它在外部特性上是一个普通函数。

3. 友元函数在实现时不要带上`friend`关键字，且不能使用类的作用域运算符`::`。

### 强制类型转换

C++11中允许类的强制类型转换，也就是说可以将：

1. 类转换为基本数据类型，分为显式转换和隐式转换。

2. 基本数据类型转换为类，分为显示转换和隐式转换。

基本数据类型转换为类时需要有构造函数支持，也就是说假设需要将一个`long`转换为一个类，那么这个类就必须拥有一个只接受一个`long`型参数的构造函数。这样的转换是隐式的，但是这样很容易犯错，因此可以加上`explicit`关键字来显示转换：

```C++
//date.h
namespace DATE{
    class Date{
    private:
        ...
    public:
        Date();
        Date(long million_seconds);
        ...
    };
}
//main.cpp
using DATE::Date;
Date d4 = 282487515;//这是隐式转换，容易出错。
```

下面是显式类型转换，不容易出错：

```C++
//date.h
namespace DATE{
    class Date{
    private:
        ...
    public:
        Date();
        explicit Date(long million_seconds);
        ...
    };
}
//main.cpp
using DATE::Date;
Date d4 = (Date)282487515;//这是显式转换
```

类也可以转换为基本数据类型，但是这也需要类实现一个成员方法，这里的例子是将类转换为`long`型，同样也有显示转换和隐式转换两种：

```C++
//date.h
namespace DATE{
    class Date{
    private:
        ...
    public:
        Date();
        Date(long million_seconds);
        ...
        explicit operator long() const;//这是C++中的强制类型转换函数
    };
}
//date.cpp
DATE::Date::operator long() const {
    return long(this->million_seconds);
}
//main.cpp
using DATE::Date;
long tmp = (long) d3;
```

下面是没有`explicit`关键字的隐式类型转换，这样比较容易出错：

```C++
//date.h
namespace DATE{
    class Date{
    private:
        ...
    public:
        Date();
        Date(long million_seconds);
        ...
        operator long() const;//这是C++中的强制类型转换函数
    };
}
//date.cpp
DATE::Date::operator long() const {
    return long(this->million_seconds);
}
//main.cpp
using DATE::Date;
long tmp = d3;
```
这个成员函数比较特殊：没有返回值。通常的形式是：`operator typeName() const;`，在实现时也要注意这种成员函数没有返回值。

## 第十二章 类和动态内存分配

本章主要讲了类内部的动态内存分配的知识点，原来以为类的动态内存分配很简单，唯一需要注意的就是分配的内存需要在析构函数中释放，现在看来这种想法太简单了，它没有考虑到变量拷贝和变量赋值的情况。本章主要讲了五个知识点，分别是拷贝构造函数，`=`运算符的重载，静态成员方法，定位`new`运算符以及成员变量初始化列表。

1. 拷贝构造函数：这种函数又叫做复制构造函数，是对象拷贝时自动调用的。对象通常在以下三种情况中会被执行拷贝：
   1. 函数参数作为值传递时，编译器会传递一个原有对象的拷贝，这时会调用对象的拷贝构造函数。
   2. 函数的返回值以值的形式返回，这时编译器也会拷贝对象的值到一个寄存器中，作为函数的返回值。
   3. 当一个对象作为另一个对象的初始化参数时：`Class a = b;`这时就会显式调用`Class`的拷贝构造函数初始化`a`。
   
   当对象拷贝时，如果对象的类中没有使用`new`运算符分配的内存，那么可以使用默认的拷贝构造函数。但是如果有`new`分配的内存，就需要自定义拷贝构造函数。这是因为默认的构造函数只是进行变量值的传递，如果类中有指针指向`new`运算符申请的内存，那么默认拷贝构造函数仅仅会拷贝原有类中指针的内容而不会复制指针指向的内存的内容，这就是浅拷贝，而拷贝构造函数需要实现的是深拷贝，也就是需要将指针指向的的内存的内容也进行拷贝。通常拷贝构造函数接受一个`const`型的引用参数。

2. `=`运算符的重载：对`=`运算符的重载只能作为类的成员函数进行，而不能作为友元函数或者普通的函数进行。通常情况下默认的`=`运算符也仅仅提供值的拷贝，和上面默认构造函数所面临的问题一样，这种浅拷贝是有缺陷的，因此如果类中拥有使用`new`运算符申请的内存时，就需要重载`=`运算符。具体需要注意的事项有以下四点：
   1. `=`运算符的重载只能作为类的成员变量，而不能作为友元函数或者普通函数。这个重载接受一个`const`类型的类的引用参数。
   2. `=`运算符的重载返回一个类的引用变量，切记不能加`const`关键字。
   3. 在进行实际的运算符重载操作之前需要判断：操作数的地址和`this`指向的内存是不是同一个，如果是则直接返回`*this`即可。
   4. 需要首先释放掉原有类中使用`new`关键字申请的内存，然后再根据要赋值的对象重新申请内存。
3. 静态成员方法：这种方法是使用`static`关键字修饰的方法，它只能使用静态成员变量，且只能通过`Class::function()`的形式调用。需要注意的是：静态成员方法在实现时和友元函数一样不能带有`static`关键字。
4. 使用定位`new`运算符申请对象的内存时不能使用`delete`关键字来释放内存，原因是定位`new`运算符和`delete`操作是不匹配的。通常情况下如果类中没有使用`new`运算符申请的内存，这样的对象可以不用管。但是如果需要让对象的析构函数执行那么就需要显式调用对象的析构函数，因为这里不能使用`delete`运算符，因此对象的析构函数不能自动执行，只能手动调用。
5. 类的变量初始化列表：类的变量初始化列表只是用在有`const`关键字修饰或者引用成员变量上，之所以有这样的设定是因为这两类变量都需要在初始化而后期不能重新赋值：
   1. 对于`const`修饰的变量来说：要么在变量声明的时候初始化，要么使用类的变量初始化列表。需要注意的是：如果这个变量是一个类则可以既可以不在变量声明时初始化，又可以不使用类的成员变量初始化列表。这是因为如果一个类有默认构造函数，那么使用`const Class a;`的意思就是调用类的默认构造函数初始化变量。
   2. 对于使用类中的引用变量来说，这种变量也需要在声明时提供初始化，或者使用类的成员变量初始化列表。但是和`const`修饰的变量不同，引用变量必须使用变量来初始化，就是说`int &a = 15;`这样是不行的，但是`const`则没有这种限制：`const int a = 15;`。这个限制在初始化引用变量时要注意，千万不能使用常量来初始化引用变量。
   3. 不仅仅对于`const`修饰和引用型变量，类中所有的变量都可以使用初始化列表的形式初始化。

### 拷贝构造函数

拷贝构造函数提供对对象的深拷贝，只要类中拥有使用`new`运算符申请的内存就都需要自定义拷贝构造函数：

```C++
//String.h
#ifndef BASICKNOWLEDGE_STRING_H
#define BASICKNOWLEDGE_STRING_H

#include <iostream>

class String {
private:
    char *p;
    int len;
    static int num_strings;
    static const int CIN_LIMIT = 60;
public:
    ...
    String(const String &obj);//拷贝构造函数
    ...
};

#endif //BASICKNOWLEDGE_STRING_H
//String.cpp
int String::num_strings = 0;
String::String(const String &obj) {
    num_strings ++;
    len = strlen(obj.p);
    p = new char[len + 1];
    strcpy(p, obj.p);
}
```
以上就是一个拷贝构造函数的例子，这个例子中需要注意的是：

1. 拷贝构造函数接受一个`const String &`型的参数。
2. 在拷贝构造函数内部需要提供深拷贝。
3. 类的成员变量中如果有`static`关键字修饰（而没有`const`关键字修饰），就需要在`cpp`文件中显式定义，而不能在声明时定义<del>static int a = 25;</del>。且这种定义必须是全局唯一的，否则会引起错误。

### 对`=`运算符重载

这个运算符的重载较为特殊，有四个需要特别注意的点：

1. `=`运算符的重载只能作为类的成员变量，而不能作为友元函数或者普通函数。这个重载接受一个`const`类型的类的引用参数。
2. `=`运算符的重载返回一个类的引用变量，切记不能加`const`关键字。
3. 在进行实际的运算符重载操作之前需要判断：操作数的地址和`this`指向的内存是不是同一个，如果是则直接返回`*this`即可。
4. 需要首先释放掉原有类中使用`new`关键字申请的内存，然后再根据要赋值的对象重新申请内存。

```C++
//String.h
#ifndef BASICKNOWLEDGE_STRING_H
#define BASICKNOWLEDGE_STRING_H

#include <iostream>

class String {
private:
    char *p;
    int len;
    static int num_strings;
    static const int CIN_LIMIT = 60;
public:
    ...
    String &operator=(const String &obj);
    ...
};

#endif //BASICKNOWLEDGE_STRING_H
//String.cpp
String& String::operator=(const String &obj) {
    if (this == &obj){
        return *this;
    }
    delete []p;
    len = strlen(obj.p);
    p = new char[len + 1];
    strcpy(p, obj.p);
    return *this;
}
```
该运算符重在和`<<`运算符重载有一些相同之处：他们都返回的是对象的非`const`引用。区别在于：`<<`运算符重载使用的是友元函数进行的，且只能使用友元函数实现。对`=`运算符的重载是用类成员函数进行的，且只能使用类成员函数。

### 静态成员方法

C++中的静态成员方法和Java中的静态成员方法相同，他们的区别仅仅在于实现上：

```C++
//String.h
#ifndef BASICKNOWLEDGE_STRING_H
#define BASICKNOWLEDGE_STRING_H

#include <iostream>

class String {
private:
    char *p;
    int len;
    static int num_strings;
    static const int CIN_LIMIT = 60;
public:
    ...
    static int how_many();
};

#endif //BASICKNOWLEDGE_STRING_H
//String.cpp
int String::how_many() {
    return num_strings;
}
```

### 类的变量初始化列表

详情见书464页，这个知识点没有进行代码练习，所有的知识点都在本章开始时声明了。

## 第十三章 类继承

本章主要讲了类继承中的知识点，这一章的知识点是比较复杂的，不过这估计也是C++中类的知识点中最复杂的一个了，一共总结出八个知识点，分别是：C++中子类的变量会覆盖父类的变量，子类方法中显示调用父类方法，父类的析构函数声明为虚函数，类中的虚函数表，C++中同名方法的覆盖，以及由此引出的返回值协变和多个同名方法的覆盖，抽象基类纯虚函数以及在类继承中使用`new`运算符。这八个点的知识总结如下：

1. C++中子类对父类变量的覆盖：如果子类定义了一个与父类相同的变量，无论这个变量是私有的，公有的或者是保护的，子类中的变量都会覆盖父类的同名变量，也就是说在子类中的方法访问这个变量，得到的将是子类中变量的值，但是如果在父类中访问这个变量，得到的又是父类中这个变量的值。

2. 子类中显示调用父类的方法：假设父类名为`super`，该类中拥有一个方法名为`func()`，那么如果这个类有一个子类，在子类的某一个方法中想要调用父类的这个方法，应该这么写：`super::func()`。

3. 虚析构函数：由于父类中可能有动态内存分配，因此定义一个析构函数是必需的。但是普通的析构函数可能会由于对象指针或者引用的关系而得不到正确的调度，因此将析构函数声明为虚函数是必要的。这是为了保证父类的析构函数得到正确的调度。

4. C++中的虚函数表：这是C++中一个非常重要的知识点，虚函数表指的是一个类中虚函数指针组成的一个数组。通常情况下，如果子类对父类的一个同名方法（方法的特征标也相同）重新定义，那么子类中的该方法就覆盖了父类的方法。举例来说：假如父类`super`中有一个方法命名为：`func()`，若子类`sub`重新定义了这个方法`func()`那么就说子类覆盖了父类的这个方法，如果用子类的实例去调用这个方法，那么得到的结果是`sub`中所定义的而不是父类中所定义的，如果通过`super`实例来调用这个方法，得到的结果是`super`中定义的。但是，如果方法的调用不是通过对象而是通过指针或者引用呢？那么情况就会变得不同：如果是一个`super`类的指针或者引用指向`sub`类的对象，那么父类指针虽然指向子类，但是如果用父类的指针或引用调用`func()`方法，得到的结果依然是父类中定义的。解决这个问题的办法就是将要重载的方法声明为`virtual`。这其中的原理就是C++中虚函数表，虚函数表解释起来较为繁琐，建议看书上的504页。

5. C++中同名方法的覆盖：如果子类重新实现了父类的某个方法（不要求特征标和返回值都相同），那么子类的这个方法就会覆盖掉父类的同名方法，无论这个方法是不是`virtual`的（当然，如果采用父类指针或引用指向子类，那么父类被覆盖的方法依然能被这样的指针或引用调用，在这种情况下，反而是子类中的方法被隐藏了）。由此引出了两个知识点：
   1. 类方法返回值协变：这是因为子类的方法会覆盖父类的同名方法，而不论方法的参数和返回值是否相同。假如父类中有一个方法的返回值是`int`，`double`或其他数据类型（也可以是用户自定义类型）。然而子类想要返回一个与父类不同类型的值，那么子类就可以利用同名方法覆盖这个特性，自己重新定义该方法，返回自己需要的类型，子类的方法就将覆盖父类的方法。
   2. 多个同名函数的覆盖问题：假如父类中有一个方法的多个重载（即存在多个函数名相同但是特征标不同的方法），子类中只要定义一个父类的同名方法（即定义一个函数名相同，特征表可以不相同的方法），那么父类中所有的相同函数名的方法都将被隐藏。解决的办法就是在子类中全部重新定义一次，如果没有新的逻辑可以直接调用父类的实现。

6. 抽象基类与纯虚函数：纯虚函数是指在虚函数生命结尾加一个`=0`的标识，纯虚函数可以实现但是没有必要。包含有纯虚函数的类叫做抽象类。抽象类不能被实例化。抽象基类被继承时：
   1. 抽象基类的子类必须负责抽象基类的构造。
   2. 抽象基类的子类必须将抽象基类中的全部纯虚函数实现才可以。
   3. 抽象基类的继承和普通类的继承规则完全一致。
 
7. 在类继承中使用`new`运算符，这里有三个关键点：
   1. 无论是父类还是子类，只要在类中有动态分配内存的行为，就需要定义复制构造函数，`=`运算符的重载以及析构函数。
   2. 子类要关心父类的内存分配情况：即假如子类中也有动态内存分配，那么子类就需要定义复制构造函数，`=`运算符的重载以及定义析构函数。子类的复制构造函数要考虑到父类的构造，可以直接采用父类的复制构造函数也可以调用父类别的构造函数；子类`=`运算符的重载需要显式调用父类赋值运算符的重载函数。
   3. 子类的析构函数不用关心父类的析构，各个析构函数只用声明为`virtual`并且释放自己类中分配的内存即可。

### 子类变量对父类变量的覆盖

子类和父类中变量的关系是比较复杂的，众所周知C++有三种访问权限，分别是`public`，`private`以及`protected`：

1. `private`变量：这种变量只能在类的内部访问，不能在子类中访问，也无法通过对象访问。

2. `protected`变量：这种变量可以在类的内部访问，也可以在继承类中访问，但是无法通过对象访问。

3. `public`变量：这种变量可以在类内部访问，也可以在子类中访问，还可以通过对象访问。

不论变量是什么访问形式，只要子类中定义了与父类中同名的变量，那么子类中的变量都将覆盖父类中的变量。当然这话总覆盖仅仅指的是对子类对象和子类方法而言的。对于使用父类指针或引用指向子类的情况或者是在父类方法中对这些变量的使用来说，这些变量又不会被隐藏：

```C++
//super.h
class super{
private:
    string pri = "super-private";
protected:
    string pro = "super-protected";
public:
    string pub = "super-public";
    void show_super(){
        cout << "pri: " << pri << ", pro: " << pro << ", pub: " << pub << endl;
    }
};
class sub: public super{
private:
    string pri = "sub-private";
protected:
    string pro = "sub-protected";
public:
    string pub = "sub-public";
    void show_sub(){
        cout << "pri: " << pri << ", pro: " << pro << ", pub: " << pub << endl;
    }
};

//main.cpp
super s1 = super();
sub s = sub();
s.show_sub();
s.show_super();
cout << "super's pri: " << s1.pub << endl;
cout << "sub's pri: " << s.pub << endl;

super *s2 = &s;
cout << "super's pri: " << s2->pub << endl;
```
输出结果如下：

```
pri: sub-private, pro: sub-protected, pub: sub-public
pri: super-private, pro: super-protected, pub: super-public
super's pri: super-public
sub's pri: sub-public
super's pri: super-public
```

知识点总结如下：

1. 子类对父类变量的覆盖仅仅在子类的作用范围内：通过子类对象调用`public`型变量，以及子类方法中对变量的使用。通过父类对象调用`public`型变量或者父类方法，或通过子类对象调用父类方法则完全不会受到变量覆盖的影响，他们依然可以使用父类中受到覆盖的变量。

2. 如果父类的指针或引用指向子类，那么这个父类指针或引用依然使用的是父类中定义的变量或者方法，除非某个方法是虚函数，否则这样的指针或引用一律调用的是父类的变量或方法。

### C++中同名函数的覆盖

同名函数的覆盖和同名变量的覆盖是非常相近的，代码如下：

```C++
//super.h
class super{
public:
    void show(){
        cout << "This is super::show()" << endl;
    }
};
class sub: public super{
public:
    void show(string name){
        super::show();
        cout << "This is sub:show()" << endl;
    }
};

//main.cpp
sub s = sub();
s.show(string("David"));//有效的调用，将输出This is sub:show()
s.show();//无效的调用，sub中的show函数覆盖了super中的
super *s1 = &s;
s1->show();//有效的调用，将输出This is super::show()
s1->show(string("David"));//无效的调用
```
由上面的代码可以看出，在同名函数的覆盖中，只要求函数名相同就能完成函数的覆盖。总结知识点如下：

1. 同名函数的覆盖仅仅作用在子类对象上，在子类方法中依然可以调用父类中被覆盖的方法。

2. 通过父类指向子类对象的指针或者引用将调用父类被覆盖的方法。

### 抽象基类和纯虚函数

这一点和Java中的完全相同，只用看代码就好：

```C++
//abc.h
#ifndef BASICKNOWLEDGE_ABC_H
#define BASICKNOWLEDGE_ABC_H
#include <string>

namespace abc_example{
    using namespace std;
    class Person {
    protected:
        enum gender{
            male,
            female
        };
    private:
        string name;
        int age;
        gender gen;
        int height;
    public:
        Person(const string &name, int age, gender gen, int height);
        virtual void show_type() const  = 0;
        virtual void show_level() = 0;
    };
    class Doctor: public Person{
    private:
        int level;
        int salary;
    public:
        Doctor(const string &name, int age, int height, int level, int salary, gender gen = male);
        virtual void show_type() const;
        virtual void show_level();
    };
}
#endif //BASICKNOWLEDGE_ABC_H

//abc.cpp
#include "abc.h"
#include <iostream>

namespace abc_example{
    Person::Person(const std::string &name, int age, abc_example::Person::gender gen, int height) {
        this->name = name;
        this->age= age;
        this->gen = gen;
        this->height = height;
    }
    Doctor::Doctor(const std::string &name, int age, int height, int level,
                   int salary, abc_example::Person::gender gen): Person(name, age, gen, height) {
        this->salary = salary;
        this->level = level;
    }
    void Doctor::show_type() const {
        cout << "type is Dcotor" << endl;
    }
    void Doctor::show_level() {
        cout << "Doctor's level is " << level << endl;
    }
}
```

总结如下：

1. 纯虚函数形如：`virtual funcname() = 0;` ,拥有纯虚函数的类被称为抽象基类。抽象基类不能被初始化，只能被继承。

2. 抽象基类的子类要负责抽象基类的构造，并且要实现抽象基类全部的纯虚函数。

### 在类继承中使用`new`运算符

这一点和普通类的动态内存分配十分相似：也是哪个类使用了动态内存分配，哪个累就想要自定义复制构造函数，重载赋值运算符和自定义析构函数。给出一个例子如下所示：

```C++
//person.h
#ifndef BASICKNOWLEDGE_PERSON_H
#define BASICKNOWLEDGE_PERSON_H

#include <cstring>
#include <iostream>
using namespace std;
class Person {
private:
    char *p;
public:
    Person(){
        this->p = new char[20];
    }
    Person(const Person &p){
        this->p = new char[20];
    }
    Person &operator=(const Person &obj){
        if (this == &obj){
            return *this;
        }
        delete []this->p;
        this->p = new char[20];
        return *this;
    }
    virtual ~Person(){
        delete []this->p;
    }
};
#endif //BASICKNOWLEDGE_PERSON_H
//manager.h
#ifndef BASICKNOWLEDGE_MANAGER_H
#define BASICKNOWLEDGE_MANAGER_H

#include "Person.h"
using namespace std;
class Manager: public Person{
private:
    char *label;
public:
    Manager(){
        this->label = new char[20];
    }
    Manager(const Manager &m): Person(m){
        this->label = new char[20];
    }
    Manager &operator=(const Manager &m){
        if (this == &m){
            return *this;
        }
        Person::operator=(m);//需要显式调用父类的赋值运算符
        delete []this->label;
        this->label = new char[20];
        strcpy(this->label, m.label);
        return *this;
    }
    virtual ~Manager(){
        delete [] this->label;
    }
};
#endif //BASICKNOWLEDGE_MANAGER_H
```
总结知识点如下：

1. 无论是父类还是子类，只要在类中有动态分配内存的行为，就需要定义复制构造函数，`=`运算符的重载以及析构函数。

2. 子类要关心父类的内存分配情况：即假如子类中也有动态内存分配，那么子类就需要定义复制构造函数，`=`运算符的重载以及定义析构函数：
   1. 子类的复制构造函数要考虑到父类的构造，可以直接采用父类的复制构造函数也可以调用父类别的构造函数；
   2. 子类`=`运算符的重载需要显式调用父类赋值运算符的重载函数。
   3. 父类的析构函数需要是虚函数，子类的析构函数不用做额外的操作。

## 第十四章 C++中的代码重用

本章主要讲C++中三块知识，分别是C++中的私有继承和保护继承，C++中的多继承和C++中的类模板。知识点是比较繁多的，一共总结出12个知识点，分别列举如下：

1. 私有继承：这种继承方式是要用来实现`has-a`关系，私有继承的特点是：父类中的保护成员和公有成员在子类中都被继承为私有成员；父类指针或引用无法隐式指向子类对象地址或者实例。这种形式的继承主要用于`has-a`关系的实现，但是这种关系大多不使用私用继承来实现，因为私有继承一方面概念较为复杂，另一方面私有继承也无法处理类中有多个同类实例的情况。

2. 保护继承：这种继承方式同样用于实现`has-a`关系，保护继承的特点是：父类中的保护成员和公有成员在子类中被继承为保护成员；父类的指针或引用无法隐式指向子类对象的地址或实例。但是保护继承和私有继承相比有一个较为特殊特殊的地方：私有继承将父类的共有和保护成员都声明为私有的，该类如果再派生出一个类，这个类就无法访问该类父类的公有成员和保护成员，但是保护继承则不然。

3. 多继承方面要注意两个问题：1. 若一个类继承自多个类，而多个类又都继承自同一个类，那么处于最底层的子类中就会有多个处于最顶层父类的对象，这在类型转换上会出现问题。2. 若一个类的多个父类都定义了同一个方法，而子类并没有定义它，当子类要调用这个方式的时候就会出现二义性，因为多个编译器都定义了这个方法，编译器将不知道调用哪个（C++中没有Python里面所谓多个父类BFS扫描的操作）。为了解决这两个在多继承中遇到的问题引出了下面三个技术：
   1. 虚基类：虚基类指的是在继承父类时使用`virtual`关键字，这时父类便称为虚基类，此处的虚和虚函数的虚没有任何关系，使用这个名词和`virtual`关键字只是为了减少概念。若多个类都虚继承自同一个父类，那么这多个子类共享同一个父类对象。在具体的写法上若类B虚继承自A则写作：`class A : virtual public B`或`class A: public virtual B`。
   
   2. 构造函数的特殊语法：如果一个类的继承树上有多个类，且其中有的类虚继承自同一个类，且这个虚基类没有被除其直接派生的类构造过，这个类就需要负责虚基类的构造，因为编译器不允许直接继承虚基类的类负责虚基类的构造（这很容易理解，因为虚基类的存在说明有多个子类共享同一个虚基类对象，如果允许其直接子类对其构造那么这个虚基类对象就会被构造多次，导致其状态的不确定性）。
   
   3. 方法的二义性：由于多继承中一个类可能有多个父类，若其父类中有不止一个对某个方法进行了定义且这个类又没有对该方法进行定义（换句话说如果要调用这个方法就需要在其父类中寻找），当使用该类的对象调用该方法时就会出现二义性（因为编译器不知道该使用父类中对该方法的哪个定义）。当然需要注意的是：出现二义性的必要条件是这些方法在“同一层级”的父类中被定义多次，所谓“同一层级”指的是在继承树中处于同一层次：假如`A`类继承自`B`和`C`，那么`B`和`C`就处于继承树的同一层级，若`B`和`C`又都继承自`D`，那么`D`的层级将高于`B`和`C`，也高于`A`。如果某个方法在距离调用对象所在类的最近层级中只被定义一次，而在更远的层级中又被定义了一次或多次，这种情况下不会引发二义性，编译器只会调用层级距离最近的那个方法。

4. 类模板的使用：类模板主要为了增加类的通用性，类模板一般用于处理容器类，总结其使用知识点为以下七个：
   
   1. 类模板的声明和实现：类模板通常需要在类声明的开始使用`template<class T>`告诉编译器该类中使用了模板`T`，在类的内部通常不需要显式注明模板`<T>`，但是在类的外部，特别是使用作用域选取运算符`::`时需要显示注明模板声明`template<class T>`和类模板`<T>`。
   
   2. 模板类的成员函数在声明时不能在单独的文件中声明，如果这样做会导致编译器找不到成员的实现。解决这个问题最简单的方法是在模板类的定义文件中实现这些成员函数。
   
   3. 模板类在使用时需要传入具体的类型，这个类型可以是整型，浮点型也可以是用户自定义类型。当然也可以传入指针类型，如`<char *>`。当传入指针类型时就显得较为特殊，意为该类主要用于管理指针，这种情况下一般在类中定义一个二重指针用于维护一个指针数组来管理指针。
   
   4. 模板声明还可以用于传递参数：`template <class T, int n>`这样的声明带有一个`int n`参数，在具体使用时可以通过模板传入参数，最明显的应用就是模板类`array`的使用，这个类在使用时就以模板参数传入了一个指示`array`大小的参数：`array<int, 5> arr = {1,2,3,4,5};`。需要注意的是：模板参数只能作为常量使用而不能修改，且模板参数只能是整形，枚举，指针和引用。也就是说`double`不能作为模板参数但是`double *`就可以。
   
   5. 模板类的具体化：具体化又可以分为三个小点：分别是类的显示具体化，显示实例化，隐式实例化：
   
      1. 隐式实例化：这种实例化就是通常使用模板类的方法。
      
      2. 显式实例化：这种实例化仅仅有语法层面的意义，没有什么实际的作用。
      
      3. 显式具体化：假如模板类在针对某个特定类型时需要重新定义处理逻辑又不想改变原有模板类，那么显式具体化技术就可以被使用。如果一个类中使用了多个模板，还需要注意部分具体化技术。
      
   6. 模板成员函数：模板成员函数是在模板类内部的成员函数上再次使用模板，这种模板成员函数在定义时有一点需要注意的：假如模板成员函数所在的类使用了模板`template<class T>`，该成员函数又使用了模板`template<class V>`，那么在实现时模板声明不能写作`template <class T, class V>`而需要写作`template<class T> template<class V>`。
   
   7. 友元函数模板：友元函数由于不是模板类的成员函数因此在使用模板时比较特殊，有非模板友元函数，被约束的模板友元函数和非约束的模板友元函数。
      
      1. 非模板友元函数：这种友元函数不使用模板。由于该友元函数不是模板函数因此可以在单独的文件中实现，如果该友元函数使用了模板类作为参数，那么这个友元函数需要对每个模板实例都定义一次：也就是说针对使用`int`模板的类需要定义一次该友元，针对`double`模板的类又需要定义一次。
      
      2. 约束的模板友元函数：这种友元函数是模板函数，因此不能在单独的文件中实现。这种类型的友元函数需要使用三步来定义：首先需要声明该友元函数：`template<class T> void fri_func(T);`；然后需要在类内部显示具体化：`friend void fri_func<>(Class);`；最后需要在类的外部实现：`template<class V> void fri_func(Class<V>){};`
      
      3. 非约束的模板友元：这种友元函数和模板成员函数的定义和实现是一模一样的，区别仅仅在于友元函数不是类的成员函数，因此在实现上不用像模板成员函数一样写两个`template`。

### 模板类管理指针

这种情况下通常是在模板类中定义一个二重指针管理一个指针数组来实现：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class T>
class StackTemp{
private:
    T *items;
public:
    StackTemp(int size){
        items = new T[size];
    }
    ~StackTemp(){
        delete []items;
    }
};
#endif //BASICKNOWLEDGE_STACKTEMP_H
```
假如现在的模板类型传入一个`<char *>`则类的声明变为：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class char *>
class StackTemp{
private:
    char * *items;
public:
    StackTemp(int size){
        items = new char *[size];
    }
    ~StackTemp(){
        delete []items;
    }
};
#endif //BASICKNOWLEDGE_STACKTEMP_H
```
如上面代码所示：管理作用一目了然。

### 模板类的显示具体化

模板类的显示具体化是为了解决模板类的通用方法无法解决特定问题时使用的技术，首先定义一个模板类：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class T>
class StackTemp{
private:
    T items;
public:
    StackTemp(int size){
        items = new T[size];
    }
    ~StackTemp(){
        delete []items;
    }
    void show();
};

template <class T>
void StackTemp<T>::show() {
    using namespace std;
    cout << "This is StackTemp<T>::show()" << endl;
}
#endif //BASICKNOWLEDGE_STACKTEMP_H
```
假如上面的类不能处理`char*`型模板，那么就需要对模板进行显示具体化，需要注意的是显示具体化时所有的成员函数都变成非模板函数，需要在单独的`cpp`文件中实现其成员函数。

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template<>//显式具体化
class StackTemp<char *>{
private:
    char *items;
public:
    StackTemp(int size){
        items = new char[size];
    }
    ~StackTemp(){
        delete []items;
    }
    void show();
};
#endif //BASICKNOWLEDGE_STACKTEMP_H

//StackTemp.cpp
void StackTemp<char *>::show() {
    using namespace std;
    cout << "This is StackTemp<char *>::show()" << endl;
}

//main.cpp
StackTemp<int> s = StackTemp<int>(10);
s.show();
StackTemp<char *> s1 = StackTemp<char *>(10);
s1.show();
```
分别输出：

```shell
This is StackTemp<T>::show()
This is StackTemp<char *>::show()
```

### 模板成员函数

举例如下：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class T>
class StackTemp{
private:
    T items;
public:
    StackTemp(int size){
        items = new T[size];
    }
    ~StackTemp(){
        delete []items;
    }
    template<class V>
    void getname(V v);
};

template <class T>
template <class V>
void StackTemp<T>::getname(V v) {
    using namespace std;
    cout << "This is StackTemp<T>::getname(V v)" << endl;
}
#endif //BASICKNOWLEDGE_STACKTEMP_H

//main.cpp
StackTemp<int> ss = StackTemp<int>(10);
ss.getname(25.4);
```
将输出：

```shell
This is StackTemp<T>::getname(V v)
```

### 模板友元

1. 非模板友元函数

这种类型的友元函数一方面不是模板类的成员函数，另一方面也不是模板函数，它也可以使用模板类作为参数，但是必须针对模板类的每一个要使用到的实例做实现，而且由于非模板友元函数不是模板函数因此需要在单独的`cpp`文件中实现：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class T>
class StackTemp{
private:
    T items;
public:
    StackTemp(int size){
        items = new T[size];
    }
    ~StackTemp(){
        delete []items;
    }
    friend void show(const StackTemp &s);
};
#endif //BASICKNOWLEDGE_STACKTEMP_H

//StackTemp.cpp
void show(const StackTemp<int> &t){
    using namespace std;
    cout << "This is friend StackTemp<int> &" << endl;
}

void show(const StackTemp<char *> &t){
    using namespace std;
    cout << "show(const StackTemp<char *> &" << endl;
}
//如果该友元函数需要处理double型那么还需要针对double的模板类定义友元函数，以此类推。
```

2. 约束的模板友元函数

这种类型的友元函数从声明到实现需要三步：

* 友元函数的声明。

* 类内部友元函数的具体化。

* 类外部友元函数的实现

具体举例如下：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class T> void show_1(T &);//模板友元的声明

template <class T>
class StackTemp{
private:
    T items;
public:
    StackTemp(int size){
        items = new T[size];
    }
    ~StackTemp(){
        delete []items;
    }
    friend void show_1<>(StackTemp<T> &);//模板友元的具体化
};

template <class T>//模板友元的实现
void show_1(StackTemp<T> &t){
    using namespace std;
    cout << "This is show_1" << endl;
}
#endif //BASICKNOWLEDGE_STACKTEMP_H
```

3. 非约束模板友元

这种类型的友元函数从声明到实现都和模板成员函数十分相似，举例如下：

```C++
//StackTemp.h
#ifndef BASICKNOWLEDGE_STACKTEMP_H
#define BASICKNOWLEDGE_STACKTEMP_H

#include <iostream>
template <class T>
class StackTemp{
private:
    T items;
public:
    StackTemp(int size){
        items = new T[size];
    }
    ~StackTemp(){
        delete []items;
    }
    template<class V>
    friend void show_2(V &v);
};

template <class V>
void show_2(V &v){
    using namespace std;
    cout << "This is show_2()" << endl;
}
#endif //BASICKNOWLEDGE_STACKTEMP_H
```

总结来说：只要是模板类或者模板函数都不能在单独的`cpp`文件中定义，除非只打算在这个`cpp`文件中使用，否则都需要在`.h`文件中定义。如果不是模板则可以在单独的`cpp`文件中定义，具体的例子是：模板显示具体化中的成员，以及非模板友元。

## 第十五章 友元、异常和其他

本章是C++学习中阶段性的一章，这一章中讲了C++中最后几个关键性的问题：友元类和友元成员函数，嵌套类，异常以及RIIT（运行阶段类型识别）。之后的章节和C++的类编程就不太相关，基本上是收尾的部分。本章的内容比较简单，因此总结也较为简单。

1. 友元类：假设`B`类是`A`类的友元类，则`B`类可以访问`A`类中所有的私有成员，包括私有成员函数以及私有变量。在`A`类中，需要对友元类`B`进行友元声明：`friend class B;`，该声明可以放在`private`中，可以放在`public`中，也可以放在`protected`中，都可以。由于`B`类中需要使用`A`类的方法或成员，这需要知道`A`类的声明细节，因此`A`类的声明必须先于`B`类。

2. 友元成员函数：通常情况下，不需要将一个类完全地声明为另一个类的友元类，这将给予友元类过大的访问权限，同时也没有必要这么做。因此可以选择将某个类的一个或几个成员函数声明为另一个类的友元成员函数。这样，声明为友元的成员函数即能访问另一个类的私有成员，没有被声明为友元成员函数的则不行。同样地，需要在类中声明友元成员函数，假设类`B`的某个成员函数声明为类`A`的友元，则需要这样声明：`friend void A::func();`即需要使用域解析操作符`::`，该声明可以放在`private`中，可以放在`public`中，也可以放在`protected`中，在声明时需要注意：定义友元成员函数的类需要在接受友元成员函数类的前面声明，同时还需要在最前方声明接受友元成员函数的类，最后友元成员函数的实现需要放在接受友元成员函数类的声明后面。

3. 嵌套类：嵌套类就是定义在类中的类，这种类的访问控制和累继承相似：如果嵌套类在一个类的`private`中，那么这个类将可以访问嵌套类，但是在派生类和类对象中则不能访问。若一个嵌套类定义在`protected`中，则类可以访问这个嵌套类，派生类也可以，但是类对象不行。若是嵌套类定义在`public`部分，则无论是类本身，派生类还是类对象都可以访问这个派生类。

4. 异常：C++中的异常和Java中比起来显得较为原始，总结出五个点：
   1. 抛出什么类型的异常，就会捕获到什么类型的异常。举例来说：假如在程序中`throw "this is a string"'`就会在相应的`try catch`语句中捕获到一个字符串：`catch (const char *s);`。当然，如果抛出的是一个异常对象，那么相应的`catch`语句中也能接受到一个异常对象。
   2. 栈解退：通常程序抛出异常，程序就会被暂停执行，调用栈就会被逐层弹出，除非遇到了处理该异常的`catch`子句，在栈解退中如果再次`throw`则会抛出相同的异常。在栈解退中需要注意：函数中的自动变量在发生异常时内存会被自动释放，但是如果一个函数中有`new`运算符申请的内存则不会，这可能会造成内存泄露，需要注意。
   3. 如果不知道函数会抛出什么类型的一场可以使用`catch(...)`或者`catch(exception &e)`来捕获所有类型的异常。需要说明的是：这里异常对象使用变量引用是因为实际使用中可能会从原生异常对象中派生出异常类，使用变量引用就能让基类引用指向子类对象。
   4. 可以通过派生`exception`对象来定义自己的异常类，需要注意的是派生时要重载`const char *what()`这个虚函数。
   5. 通常如果程序遇到未捕获异常或者意外异常都会触发程序`abort()`，如果希望改变程序的这一设定，可以通过调用`set_terminate()`或`set_unexcept()`函数来修改这一默认行为。这两个函数的使用需要导入`#include<exception>`，且这两个函数接受一个函数指针，用户可以自定义。

5. RIIT：所谓RTTI即是运行时类型识别，这也开始类型强制转换的一种，不过通常是执行对引用和指针的转换。主要需要掌握三个点：
   1. `dynamic_cast`：这是动态类型转换的意思，假如要对一个引用或者指针类型进行转换可以用它来进行，当然转换的时候还是要遵循基类指针或引用可以指向子类对象的地址或对象，但是不能反过来这一设定。如果转换不成功，若是转换指针则指针会指向`NULL`，若是引用转换则会抛出一个`bad_cast`异常。
   2. `type_id`：这也是一个运算，需要引入头文件`#include<typeinfo>`，这个操作（函数）的参数可以使类也可以是对象或值为对象的表达式。它的作用是判断一个对象的或类的类型。返回一个`typeinfo`对象，该对象包含类型的全部信息，可用于判断两个对象是否属于同一个类。
   3. `typeinfo`：这是一个对象，详细的定义及用法见`type_id`。

### 友元类

友元类定义和使用时主要需要注意友元类和接受友元类的类的声明位置，这里给出一个例子，这个例子是模板类的友元类：

```C++
//TV.h
#ifndef BASICKNOWLEDGE_TV_H
#define BASICKNOWLEDGE_TV_H

#include <iostream>

template <class T> class TV {
public:
    friend class Remote;
private:
    void getname();
};
template <class T> void TV<T>::getname() {
    using namespace std;
    cout << "This is TV::getname()" << endl;
}
class Remote{
public:
    template <class T> void friendfunc(TV<T> &tv);
};

template <class T> void Remote::friendfunc(TV<T> &tv) {
    tv.getname();
}

#endif //BASICKNOWLEDGE_TV_H
```

需要注意的是：这个例子中由于使用了类模板因此在实现类成员函数的时候不能再单独的文件中进行，这并不是友元类的限制。

### 友元成员函数

如果一个类中仅有一个或者几个方法需要使用别的类的私有成员，那么就可以使用友元成员函数技术。该技术举例如下，这个例子同样是使用模板类来举例，例子中主要需要注意各个类的声明位置以及类的提前声明：

```C++
//TV.h
#ifndef BASICKNOWLEDGE_TV_H
#define BASICKNOWLEDGE_TV_H

#include <iostream>

template <class T> class TV;

class Remote_1{
public:
    template <class T> void func(TV<T> &tv);
};

template <class T> class TV {
public:
    template <class V>friend void Remote_1::func(TV<V> &tv);
private:
    void getname();
};

template <class T> void TV<T>::getname() {
    using namespace std;
    cout << "This is TV::getname()" << endl;
}

template <class T> void Remote_1::func(TV<T> &tv) {
    tv.getname();
}

#endif //BASICKNOWLEDGE_TV_H

```